In [ ]:
# DRAMP3_Crawbing
import openpyxl
import pandas as pd
import requests  # 导入requests包
from lxml import etree
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning

wb = openpyxl.Workbook()
sheet1 = wb.active
sheet1.append(['dramp', 'name',  'cyclic', 'source', 'family', 'Gene', 'seq', 'seq_stapling', 'N_ter', 'C_ter', 'Structure', 'Structure_des',
               'function', 'function_detail', 'PDB', 'Formula', 'Absent_AA', 'Mass', 'PI','Hydrophobic_Residues', 'Net_Charge', 'Boman_Index',
               'Hydrophobicity', 'Aliphatic_Index', 'Half_Life', 'Hemolytic_Activity', 'Cytotoxicity', 'target', 'target_organism', 
               'Absorbance_280nm', 'Polar_Residues', 'pubmed_id', 'uniprotID'])
sheet2 = wb.create_sheet('title')
sheet2.title = 'new sheet'
sheet2.append(['lost'])
sheet1.sheet_properties.tabColor = '000000'
sheet2.sheet_properties.tabColor = '000000'
disable_warnings(InsecureRequestWarning)
if __name__ == '__main__':
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54 "
    }

    url_main = 'http://dramp.cpu-bioinfor.org/browse/All_Information.php?id='
    # keyword = pd.read_excel('fileName.xlsx', dtype=object).loc[:, 'Column_Name']
    keyword = ['DRAMP00866','DRAMP00867','DRAMP00868','DRAMP00869','DRAMP00870','DRAMP00871','DRAMP00872']# Test data
    for i in range(len(keyword)):
        Gene, seq, seq_stapling, cyclic, N_ter, C_ter, Structure, Structure_des, PDB = '', '', '', '', '', '', '', '', ''
        name, Formula, Absent_AA, Mass, PI, Hydrophobic_Residues, Net_Charge = '', '', '', '', '', '', ''
        Boman_Index, Hydrophobicity, Aliphatic_Index, Half_Life, Absorbance_280nm, Polar_Residues = '', '', '', '', '', ''
        url = url_main + keyword[i]
        response1 = requests.get(url=url, headers=headers, verify=False)
        response1.encoding = response1.apparent_encoding
        label = etree.HTML(response1.text)
        source = '##'.join(label.xpath('//*[@id="general_source"]/../following-sibling::li[1]//text()'))
        family = '##'.join(label.xpath('//*[@id="general_family"]/../following-sibling::li[1]/text()'))
        Gene = '##'.join(label.xpath('//*[@id="general_gene"]/../following-sibling::li[1]//text()'))
        target = '##'.join(label.xpath('//*[@id="activity-binding"]/../following-sibling::li[1]/text()'))
        seq = ''.join(label.xpath('//*[@id="general_sequence"]/../following-sibling::li[1]/text()')[0])
        seq_stapling = '##'.join(label.xpath('//*[@id="structure-other-mod"]/../following-sibling::li[1]/text()'))
        
        pubmed_id = '##'.join(label.xpath('//*[@id="literature-pubmed"]/../following-sibling::li[1]//text()'))
        function = ''.join(label.xpath('//*[@id="activity-biological"]/../following-sibling::li[1]/text()')).strip()
        targetOrganism_list = label.xpath('//*[@id="activity-target"]/../following-sibling::li[1]/ul[1]/li')
        target_organism = ''
        for li in targetOrganism_list:
            target_organism = ''.join(li.xpath('./b/text()')).strip() + target_organism
        HemolyticActivity_list = label.xpath('//*[@id="activity-hemolytic"]/../following-sibling::li[1]/ul[1]/li')
        Hemolytic_Activity = ''
        for li in HemolyticActivity_list:
            Hemolytic_Activity = ''.join(li.xpath('./b/text()')).strip() + Hemolytic_Activity
        Cytotoxicity_list = label.xpath('//*[@id="activity-cytotoxicity"]/../following-sibling::li[1]/ul[1]/li')
        Cytotoxicity = ''
        for li in Cytotoxicity_list:
            Cytotoxicity = ''.join(li.xpath('./b/text()')).strip() + Cytotoxicity
        target = ''.join(label.xpath('//*[@id="activity-binding"]/../following-sibling::li[1]/text()')).strip()
        function_detail = ''.join(
            label.xpath('//*[@id="comments-Function"]/../following-sibling::li[1]/text()')).strip()
        cyclic = "".join(label.xpath('//*[@id="structure-lcb"]/../following-sibling::li[1]/text()')).strip()
        uniprotID = "".join(label.xpath('//*[@id="general_swiss"]/../following-sibling::li[1]//text()')).strip()
        N_ter = "".join(label.xpath('//*[@id="structure-Nter-mod"]/../following-sibling::li[1]/text()')).strip()
        C_ter = "".join(label.xpath('//*[@id="structure-Cter-mod"]/../following-sibling::li[1]/text()')).strip()
        Structure = "".join(label.xpath('//*[@id="structure-info"]/../following-sibling::li[1]/text()')).strip()
        Structure_des = "".join(
            label.xpath('//*[@id="structure-description"]/../following-sibling::li[1]/text()')).strip()
        PDB = "".join(label.xpath('//*[@id="structure-pdb"]/../following-sibling::li[1]//text()')).strip()
        div1_list = label.xpath('//*[@id="physical"]/../following-sibling::div[1]/div[1]/ul/li/ul')
        div2_list = label.xpath('//*[@id="physical"]/../following-sibling::div[1]/div[2]/ul/li/ul')
        for ul in div1_list:
            name = "".join(ul.xpath('li[1]/h4/text()')).strip()
            Formula = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Formula' else Formula
            Absent_AA = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Absent Amino Acids' else Absent_AA
            Mass = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Mass' else Mass
            PI = "".join(ul.xpath('li[2]/text()')).strip() if name == 'PI' else PI
            Hydrophobic_Residues = "".join(
                ul.xpath('li[2]/text()')).strip() if name == 'Hydrophobic Residues' else Hydrophobic_Residues
            Net_Charge = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Net Charge' else Net_Charge
        for ul in div2_list:
            name = "".join(ul.xpath('li[1]/h4/text()')).strip()
            Boman_Index = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Boman Index' else Boman_Index
            Hydrophobicity = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Hydrophobicity' else Hydrophobicity
            Aliphatic_Index = "".join(
                ul.xpath('li[2]/text()')).strip() if name == 'Aliphatic Index' else Aliphatic_Index
            Half_Life = "".join(ul.xpath('li[2]//text()')).strip() if name == 'Half Life' else Half_Life
            Absorbance_280nm = "".join(
                ul.xpath('li[2]/text()')).strip() if name == 'Absorbance 280nm' else Absorbance_280nm
            Hydrophobic_Residues = "".join(
                ul.xpath('li[2]/text()')).strip() if name == 'Hydrophobic Residues' else Hydrophobic_Residues
            Polar_Residues = "".join(ul.xpath('li[2]/text()')).strip() if name == 'Polar Residues' else Polar_Residues
        xx_info = [keyword[i], name, cyclic, source, family, Gene, seq, seq_stapling, N_ter, C_ter, Structure, Structure_des, function, 
                   function_detail, PDB, Formula, Absent_AA, Mass, PI,Hydrophobic_Residues, Net_Charge, Boman_Index, Hydrophobicity, 
                   Aliphatic_Index, Half_Life, Hemolytic_Activity, Cytotoxicity, target, target_organism, Absorbance_280nm, Polar_Residues,
                   pubmed_id, uniprotID]
        sheet1.append(xx_info)
        print(xx_info)
    wb.save('dramp_Info.xlsx')
